#Ecuadorian Network
Evaluation of data from Pypsa-Earth


In [ ]:
"""
Setup environment and load the base PyPSA-Earth network for a specified country.
"""

import os
import sys
import warnings
import pypsa
import warnings
from pathlib import Path
import pypsa
import numpy as np
from pathlib import Path
import os
import shutil
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import requests
from matplotlib.colors import LinearSegmentedColormap, to_hex
import copy
from os.path import join

# Import all dirs
parent_dir = Path(os.getcwd()).parents[0]
sys.path.append(str(parent_dir))
from src.paths import all_dirs

dirs = all_dirs()


import logging
from pathlib import Path

LOG_FILE = join(parent_dir, "logs.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.FileHandler(LOG_FILE, encoding="utf-8")],
)


# Suppress warnings
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=UserWarning)
# Suppress unnecessary warnings for cleaner output
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


# Define country parameters
country_code = "EC"  # ISO 2-letter code (e.g., 'GH' for Ghana, 'CO' for Colombia)
country_name = "Ecuador"  # Country name
country_gadm = "ECU"  # ISO 3-letter GADM code

# Load the base network file path
network_dir = dirs["data/raw/networks"]
network_files = [
    "network_original",
    "network_snapped",
    "network_expanded",
    "network_expanded_no_orphans",
    "network_nuclear",
    "network_prod_mix",
    "network_base",
]


networks_dict = {
    nf: pypsa.Network(join(network_dir, f"{nf}.nc")) for nf in network_files
}

INFO:pypsa.io:Imported network base.nc has buses, lines, transformers


Network loaded successfully from: c:\Repositories\Repos\pypsa-earth-project\EcuadorElectricGrid\data\raw\networks\base.nc


#### Mapping buses and place of interest

In [ ]:
# ---------------------------------------------------------------------
# GADM boundary data (expects `country_gadm` to be defined upstream, e.g., "ECU")
# ---------------------------------------------------------------------

# Where to put / find GADM (expects dirs["data/raw/gadm"])
COUNTRY_ISO3 = "ECU"
GADM_BASE_DIR = None  # will be inferred from dirs dict below
GADM_VERSION = "4.1"
GADM_FILE_STEM = f"gadm41_{COUNTRY_ISO3}"
GADM_LAYER_L1 = "ADM_ADM_1"
GADM_URL = f"https://geodata.ucdavis.edu/gadm/gadm{GADM_VERSION}/gpkg/{GADM_FILE_STEM}.gpkg"

def ensure_gadm_file(gadm_path: str) -> str:
    """Ensure GADM gpkg exists locally; download if missing. Return path to gpkg."""
    gpkg_path = os.path.join(gadm_path, f"{GADM_FILE_STEM}.gpkg")
    gpkg_dir = Path(gpkg_path).parent
    if not Path(gpkg_path).is_file():
        gpkg_dir.mkdir(parents=True, exist_ok=True)
        resp = requests.get(GADM_URL, stream=True, timeout=300)
        resp.raise_for_status()
        with open(gpkg_path, "wb") as f:
            shutil.copyfileobj(resp.raw, f)
    else:
        print(f"GADM file already exists: {gpkg_path}")
    assert Path(gpkg_path).is_file(), f"GADM file not found or failed to download: {gpkg_path}"
    return gpkg_path

GADM_inputfile_gpkg = ensure_gadm_file(dirs["data/raw/gadm"])


GADM file already exists: c:\Repositories\Repos\pypsa-earth-project\EcuadorElectricGrid\data\raw\gadm\gadm41_ECU.gpkg


In [ ]:
def plot_buses_and_lines_by_voltage(network: pypsa.Network, title= None, save_name=None):
    """
    Plot Ecuadorian transmission buses/lines colored by nominal voltage (continental Ecuador only)
    and annotate each bus with its Bus ID (index).
    """

    # ---------------- Fixed colors by voltage (unique and stable) ----------------
    VOLTAGE_COLORS = {
        46.0: "#e41a1c",   # red
        48.0: "#ff7f00",   # orange
        69.0: "#4daf4a",   # green
        138.0: "#377eb8",  # blue
        145.0: "#984ea3",  # purple
        230.0: "#a65628",  # brown
        500.0: "#00008B",  # dark blue
    }
    DEFAULT_COLOR = "#999999"

    def map_voltage_color(v):
        # robust float comparison
        for k, c in VOLTAGE_COLORS.items():
            if np.isclose(v, k, atol=1e-3):
                return c
        return DEFAULT_COLOR

    # widths scale consistently with voltage
    def map_voltage_width(v):
        return v * 0.004 + 0.5

    # ---------------- Figure ----------------
    fig, ax = plt.subplots(figsize=(20, 16), subplot_kw={"projection": ccrs.PlateCarree()})

    # ---------------- Boundaries & continental extent ----------------
    adm1 = gpd.read_file(GADM_inputfile_gpkg, layer="ADM_ADM_1")
    name_cols = [c for c in adm1.columns if "NAME_1" in c or c.lower() in ("name_1", "name1", "provincia", "province")]
    if name_cols:
        name_col = name_cols[0]
        adm1_cont = adm1[~adm1[name_col].str.contains("Galápagos", case=False, na=False)].copy()
    else:
        adm1_cont = adm1
    adm1_cont.boundary.plot(ax=ax, linewidth=0.3, color="black")

    # Continental Ecuador bbox (approx; stable + fast)
    minx, miny, maxx, maxy = (-81.1, -4.8, -75.1, 1.7)
    pad_x = (maxx - minx) * 0.03
    pad_y = (maxy - miny) * 0.03

    # ---------------- Plot network ----------------
    bus_colors = network.buses.v_nom.map(map_voltage_color)
    line_colors = network.lines.v_nom.map(map_voltage_color)
    line_widths = network.lines.v_nom.map(map_voltage_width)

    network.plot(
        ax=ax,
        bus_colors=bus_colors,
        line_colors=line_colors,
        line_widths=line_widths,
        bus_sizes=0.01 / 5,
        color_geomap=True,
    )

    # ---------------- Annotate bus IDs ----------------
    coord_cols = ("x", "y") if {"x", "y"}.issubset(network.buses.columns) else ("lon", "lat")
    for bus_id, row in network.buses.iterrows():
        x_val, y_val = row[coord_cols[0]], row[coord_cols[1]]
        if np.isnan(x_val) or np.isnan(y_val):
            continue
        ax.text(
            x_val, y_val,
            str(bus_id),
            fontsize=6,
            ha="left", va="bottom",
            transform=ccrs.PlateCarree(),
            color="black",
            path_effects=[pe.withStroke(linewidth=1.5, foreground="white")],
            zorder=10,
        )

    # ---------------- Legend ----------------
    # Show only voltages that exist in the network, ordered ascending
    present_voltages = sorted(
        {k for k in VOLTAGE_COLORS.keys() if np.any(np.isclose(network.buses.v_nom.values, k, atol=1e-3))}
    )
    # If none matched (unlikely), fall back to all keys
    if not present_voltages:
        present_voltages = sorted(VOLTAGE_COLORS.keys())

    handles = [
        plt.Line2D(
            [0], [0], marker="o", linestyle="none",
            markerfacecolor=VOLTAGE_COLORS[v], markeredgecolor="none",
            markersize=10, label=f"{int(v)} kV" if float(v).is_integer() else f"{v} kV"
        )
        for v in present_voltages
    ]
    ax.legend(
        handles=handles,
        title="Nominal Voltage",
        loc="upper left",
        bbox_to_anchor=(1.1, 1),
        borderaxespad=0,
    )
    ax.set_title("Buses by Nominal Voltage (Continental Ecuador)")
    ax.set_extent([minx - pad_x, maxx + pad_x, miny - pad_y, maxy + pad_y], crs=ccrs.PlateCarree())

    if save_name:
        FIGS_DIR = dirs["results/graphs"]
        os.makedirs(FIGS_DIR, exist_ok=True)
        out = os.path.join(FIGS_DIR, save_name)
        plt.savefig(out, dpi=600, bbox_inches="tight")
    #plt.show()

In [ ]:
plot_buses_and_lines_by_voltage(
    network=networks_dict["network_original"],
    title= "Buses by Nominal Voltage (Continental Ecuador)\nRaw Pypsa-Earth Data",
    save_name= "01_EC_Grid_Original"
)

Plot only the expansion

In [ ]:
n_only_exp = copy.deepcopy(n_exp)

buses_exp = n_only_exp.buses[n_only_exp.buses.index.astype(str).str.isnumeric() &
                             (n_only_exp.buses.index.astype(int) >= 500)]
lines_exp = n_only_exp.lines[n_only_exp.lines.index.astype(str).str.isnumeric() &
                             (n_only_exp.lines.index.astype(int) >= 500)]
n_only_exp.buses = buses_exp 
n_only_exp.lines = lines_exp


plot_buses_and_lines_by_voltage(n_only_exp, save_name="ecuador_buses_only_expansion.png")

In [ ]:
n_only_exp = copy.deepcopy(n_exp)
buses_500 = n_only_exp.buses[n_only_exp.buses.v_nom >= 120] 
lines_500 = n_only_exp.lines[n_only_exp.lines.v_nom >= 120] 
n_only_exp_HV = n_only_exp.copy()
n_only_exp_HV.buses = buses_500 
n_only_exp_HV.lines = lines_500
plot_buses_and_lines_by_voltage(n_only_exp_HV, save_name="ecuador_buses_expanded_HV.png")

n_only_exp_HV.lines.to_csv("ecuador_lines_expanded_HV.csv")
n_only_exp_HV.buses.to_csv("ecuador_buses_expanded_HV.csv")



In [ ]:
#Sanity + topology checks for expansion elements against the existing network
from src.ec_network_eval import evaluate_network

n_exp, issues = evaluate_network(
    n_exp,

    data_dir = dirs["data/raw/networks"],
    downstream_path = (500,48))

for k,v in issues:
    if "empty" not in k:
        print(k)
        display(v)


orphan_buses_lines_trafos_only


,Bus,v_nom,lon,lat
109,109,48.0,-78.7385,-2.9038
122,122,69.0,-78.7498,-1.7366
134,134,69.0,-78.7659,-3.8451
135,135,69.0,-79.9708,-3.5235
168,168,69.0,-80.5068,-0.8724
172,172,69.0,-80.1647,-0.8570
180,180,69.0,-78.7958,1.2621
187,187,69.0,-80.0950,-0.8791
193,193,69.0,-79.6392,-3.7076
203,203,69.0,-80.2042,-2.3207


bus_voltage_levels


[48, 69, 138, 230, 500]

trafo_pairs_found_network


[(48, 138), (69, 138), (69, 230), (69, 500), (138, 230), (230, 500)]

topology_summary


{'230kV': {'n_components': 63, 'sizes': [5, 4, 3, 2, 2]},
 '500kV': {'n_components': 13, 'sizes': [6, 1, 1, 1, 1]}}

In [ ]:
import copy
import pandas as pd
import numpy as np

def clean_orphan_buses(n_exp, issues, log_csv_path="removed_orphan_buses.csv"):
    """
    Remove buses that are truly unreferenced by any component in the network.
    - Deep-copies the input network (does not modify original).
    - Uses orphan candidates from issues if present.
    - Verifies true orphans by scanning all component bus references.
    - Optionally writes removed buses to CSV (set log_csv_path=None to skip)
    Returns
    - n_exp_clean: deep-copied and cleaned network
    """
    # 1) Extract candidate orphan buses from issues (accepts multiple keys)
    orphan_df = None
    for key, data in issues:
        if key in ("orphan_buses", "orphan_buses_lines_trafos_only"):
            orphan_df = data
            break

    candidate_orphans = pd.Index([])
    if orphan_df is not None and "Bus" in orphan_df.columns:
        candidate_orphans = pd.Index(orphan_df["Bus"].tolist())

    # 2) Collect all buses referenced by any component
    def buses_referenced_by_components(n):
        refs = set()
        comps = [
            ("loads", n.loads),
            ("generators", n.generators),
            ("storage_units", n.storage_units),
            ("stores", n.stores),
            ("links", n.links),
            ("lines", n.lines),
            ("transformers", n.transformers),
            ("shunt_impedances", n.shunt_impedances),
        ]
        for _, df in comps:
            if df is None or getattr(df, "empty", True):
                continue
            # Any column that starts with 'bus' is considered a bus reference
            bus_cols = [c for c in df.columns if c.startswith("bus")]
            if not bus_cols:
                continue
            for col in bus_cols:
                # Normalize to string IDs, drop NaNs
                refs |= set(df[col].dropna().astype(str))
        return refs

    ref_buses = buses_referenced_by_components(n_exp)
    # Normalize the network bus index to string for cross-compatibility
    bus_index_str = n_exp.buses.index.astype(str)
    # Buses not referenced by any component
    true_orphans_idx = n_exp.buses.loc[~bus_index_str.isin(ref_buses)].index

    # 3) Intersect with candidates (if provided)
    if len(candidate_orphans) > 0:
        candidate_orphans = pd.Index(candidate_orphans)
        true_orphans_idx = candidate_orphans.intersection(true_orphans_idx)

    # 4) Deep-copy the network and remove the confirmed orphan buses
    n_exp_clean = copy.deepcopy(n_exp)
    for b in true_orphans_idx:
        n_exp_clean.remove("Bus", b)

    # 5) Optional: log removed buses
    if log_csv_path:
        pd.DataFrame({"removed_bus": pd.Index(true_orphans_idx)}).to_csv(
            log_csv_path, index=False
        )

    # 6) Sanity assert
    for b in true_orphans_idx:
        assert b not in n_exp_clean.buses.index, f"Bus {b} was not removed properly."
    logging.info("All orphan buses cleared")

    return n_exp_clean

In [ ]:
n_exp_clean = clean_orphan_buses(n_exp, issues)

INFO:root:All orphan buses cleared


Now, we need three network objects
1) One containing only the original network structure, that is, no buses, trafos, or lines that represent the expansion
2) One that contians the network structure as per the master plan
3) One that contians the nuclear buses 

The final network is the one already containing the nuclear option, so we remove from there




In [ ]:
network_nuclear = copy.deepcopy(n_exp_clean)


Remove the nuclear buses and related components to get only the "productive mix" network

In [ ]:
import copy
import logging
import pandas as pd
from os.path import join

def remove_nuclear_assets(network_nuclear, dirs, buses_csv="EC_buses_expansion.csv"):
    """
    Deep-copy the network, then remove:
    - all buses whose Name contains 'nuclear' (case-insensitive) from the expansion buses CSV
    - all lines/transformers connected to those buses

    Logging:
    - Logs the nuclear bus IDs found
    - Logs the line and transformer IDs that are removed

    Returns
    - network_prod_mix: cleaned network (deep copy of input)
    """
    network_prod_mix = copy.deepcopy(network_nuclear)

    # 1) Nuclear bus IDs (robust to NaNs/whitespace; read from expansion CSV)
    bus_csv_path = join(dirs["data/raw/networks"], buses_csv)
    bus_new = pd.read_csv(bus_csv_path)
    mask_nuc = (
        bus_new["Name"].astype(str).str.strip().str.lower().str.contains("nuclear", na=False)
    )
    ids_bus_nuclear = (
        pd.to_numeric(bus_new.loc[mask_nuc, "Bus"], errors="coerce")
        .dropna()
        .astype("Int64")
    )
    logging.info(f"nuclear buses: {ids_bus_nuclear.dropna().tolist()}")

    # 2) Work on full line/trafo tables from the deep-copied network
    lines_existing = network_prod_mix.lines.copy()
    trafo_existing = network_prod_mix.transformers.copy()

    # 3) Align dtypes for membership tests using auxiliary numeric columns
    def _add_numeric_bus_cols(df):
        if df is None or getattr(df, "empty", True):
            return df
        for col in ("bus0", "bus1"):
            if col in df.columns:
                df[col + "_num"] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
        return df

    lines_existing = _add_numeric_bus_cols(lines_existing)
    trafo_existing = _add_numeric_bus_cols(trafo_existing)

    # 4) Filter lines and transformers touching nuclear buses
    ids_lines_nuclear = []
    if lines_existing is not None and not lines_existing.empty:
        mask_lines = (
            lines_existing.get("bus0_num").isin(ids_bus_nuclear)
            | lines_existing.get("bus1_num").isin(ids_bus_nuclear)
        )
        ids_lines_nuclear = lines_existing.index[mask_lines.fillna(False)].tolist()

    ids_trafo_nuclear = []
    if trafo_existing is not None and not trafo_existing.empty:
        mask_trafos = (
            trafo_existing.get("bus0_num").isin(ids_bus_nuclear)
            | trafo_existing.get("bus1_num").isin(ids_bus_nuclear)
        )
        ids_trafo_nuclear = trafo_existing.index[mask_trafos.fillna(False)].tolist()

    logging.info(f"lines hitting nuclear buses: {ids_lines_nuclear}")
    logging.info(f"trafos hitting nuclear buses: {ids_trafo_nuclear}")

    # 5) Remove lines and transformers first, then buses (string IDs for safety)
    i_l, i_t, i_b = 0,0,0
    for l in ids_lines_nuclear:
        network_prod_mix.remove("Line", str(l))
        i_l+=1
    for t in ids_trafo_nuclear:
        network_prod_mix.remove("Transformer", str(t))
        i_t+=1
    for b in ids_bus_nuclear.dropna().tolist():
        network_prod_mix.remove("Bus", str(int(b)))
        i_b+=1
    logging.info(f"{i_l} Lines removes")
    logging.info(f"{i_t} Transformers removed")
    logging.info(f"{i_b} Buses removed")
    logging.info("Finished removing nuclear assets.")
    return network_prod_mix

network_prod_mix = remove_nuclear_assets(network_nuclear, dirs)



INFO:root:nuclear buses: [517, 518]
INFO:root:lines hitting nuclear buses: ['504', '505']
INFO:root:trafos hitting nuclear buses: ['513', '514']
INFO:root:2 Lines removes
INFO:root:2 Transformers removed
INFO:root:2 Buses removed
INFO:root:Finished removing nuclear assets.


In [ ]:
def remove_expansion_assets(
    network_in,
    dirs,
    *,
    buses_csv="EC_buses_expansion.csv",
    lines_csv="EC_lines_expansion.csv",
    trafos_csv="EC_trafo_expansion.csv",
    bus_id_col="Bus",
    line_id_col="ID",
    trafo_id_col="ID",
    ):
    """
    Deep-copy the network, then remove expansion assets:
    - Buses listed in buses_csv (column bus_id_col)
    - Lines listed in lines_csv (column line_id_col)
    - Transformers listed in trafos_csv (column trafo_id_col)

    Logging:
    - Logs IDs found in each CSV
    - Logs how many of each component were removed

    Returns
    - network_base: cleaned network (deep copy of input)
    """
    network_base = copy.deepcopy(network_in)

    base_dir = dirs["data/raw/networks"]
    bus_path = join(base_dir, buses_csv)
    line_path = join(base_dir, lines_csv)
    trafo_path = join(base_dir, trafos_csv)

    # Read CSVs
    bus_df = pd.read_csv(bus_path)
    line_df = pd.read_csv(line_path)
    trafo_df = pd.read_csv(trafo_path)

    # Extract IDs, coerce as needed
    bus_ids = pd.Series([], dtype=object)
    if bus_id_col in bus_df.columns:
        # buses often numeric in expansion files; keep robust casting
        bus_ids = pd.to_numeric(bus_df[bus_id_col], errors="coerce").dropna().astype("Int64")

    line_ids = pd.Series([], dtype=object)
    if line_id_col in line_df.columns:
        line_ids = line_df[line_id_col].dropna().astype(str)

    trafo_ids = pd.Series([], dtype=object)
    if trafo_id_col in trafo_df.columns:
        trafo_ids = trafo_df[trafo_id_col].dropna().astype(str)

    logging.info(f"expansion buses to remove (count={len(bus_ids)}): {bus_ids.dropna().tolist()}")
    logging.info(f"expansion lines to remove (count={len(line_ids)}): {line_ids.tolist()}")
    logging.info(f"expansion transformers to remove (count={len(trafo_ids)}): {trafo_ids.tolist()}")

    # Remove lines and transformers first, then buses
    removed_lines = 0
    for l in line_ids.tolist():
        try:
            network_base.remove("Line", str(l))
            removed_lines += 1
        except Exception:
            pass

    removed_trafos = 0
    for t in trafo_ids.tolist():
        try:
            network_base.remove("Transformer", str(t))
            removed_trafos += 1
        except Exception:
            pass

    removed_buses = 0
    for b in bus_ids.dropna().tolist():
        try:
            network_base.remove("Bus", str(int(b)))
            removed_buses += 1
        except Exception:
            # fall back to raw string if needed
            try:
                network_base.remove("Bus", str(b))
                removed_buses += 1
            except Exception:
                pass

    logging.info(f"{removed_lines} Lines removed")
    logging.info(f"{removed_trafos} Transformers removed")
    logging.info(f"{removed_buses} Buses removed")
    logging.info("Finished removing expansion assets.")

    return network_base

In [ ]:
network_base = remove_expansion_assets(n_prod_mix, dirs)

INFO:root:expansion buses to remove (count=23): [500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 601, 664, 690, 697]
INFO:root:expansion lines to remove (count=16): ['501', '502', '503', '504', '505', '506', '507', '508', '509', '510', '511', '512', '513', '601', '602', '603']
INFO:root:expansion transformers to remove (count=15): ['501', '502', '503', '504', '505', '506', '507', '508', '509', '510', '511', '512', '513', '514', '601']
INFO:root:14 Lines removed
INFO:root:13 Transformers removed
INFO:root:21 Buses removed
INFO:root:Finished removing expansion assets.


In [ ]:
#compare the shape of the three networks for buses, lines and trafos
import pandas as pd

# Define a helper to get basic network element counts
def net_summary(net, name):
    return {
        "Network": name,
        "Buses": len(net.buses),
        "Lines": len(net.lines),
        "Transformers": len(net.transformers),
        "Generators": len(net.generators),
        "Loads": len(net.loads),
        "Links": len(net.links)
    }

# Build a comparison table
summary = pd.DataFrame([
    net_summary(network_base, "Base"),
    net_summary(network_prod_mix, "Productive Mix"),
    net_summary(network_nuclear, "Nuclear")
])

# Display the result
print(summary)


          Network  Buses  Lines  Transformers  Generators  Loads  Links
0            Base    296    247            62           0      0      0
1  Productive Mix    317    261            75           0      0      0
2         Nuclear    319    263            77           0      0      0


In [ ]:
#drop them as nc objects so they can be loaded later
# Save networks as NetCDF files (recommended PyPSA format)
import os

output_path = dirs["data/processed/networks"]

# Ensure the directory exists
os.makedirs(output_path, exist_ok=True)

# Save each network to NetCDF in the output folder
network_base.export_to_netcdf(os.path.join(output_path, "network_base.nc"))
network_prod_mix.export_to_netcdf(os.path.join(output_path, "network_prod_mix.nc"))
network_nuclear.export_to_netcdf(os.path.join(output_path, "network_nuclear.nc"))

print("exported")



INFO:pypsa.io:Exported network 'network_base.nc' contains: transformers, buses, lines
INFO:pypsa.io:Exported network 'network_prod_mix.nc' contains: lines, buses, transformers
INFO:pypsa.io:Exported network 'network_nuclear.nc' contains: transformers, buses, lines


exported
